# **School Record**

In [1]:
import datetime
from tkinter import *
import tkinter.messagebox as mb
from tkinter import ttk
from tkcalendar import DateEntry  # pip install tkcalendar
import sqlite3

# Creating the universal font variables
headlabelfont = ("Noto Sans CJK TC", 15, 'bold')
labelfont = ('Garamond', 14)
entryfont = ('Garamond', 12)

# Connecting to the Database where all information will be stored
connector = sqlite3.connect('SchoolManagement.db')
cursor = connector.cursor()

connector.execute(
"CREATE TABLE IF NOT EXISTS SCHOOL_MANAGEMENT (STUDENT_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, NAME TEXT, EMAIL TEXT, PHONE_NO TEXT, GENDER TEXT, DOB TEXT, STREAM TEXT)"
)

# Creating the functions
def reset_fields():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    for i in ['name_strvar', 'email_strvar', 'contact_strvar', 'gender_strvar', 'stream_strvar']:
        exec(f"{i}.set('')")
    dob.set_date(datetime.datetime.now().date())


def reset_form():
    global tree
    tree.delete(*tree.get_children())

    reset_fields()


def display_records():
    tree.delete(*tree.get_children())

    curr = connector.execute('SELECT * FROM SCHOOL_MANAGEMENT')
    data = curr.fetchall()

    for records in data:
        tree.insert('', END, values=records)


def add_record():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    name = name_strvar.get()
    email = email_strvar.get()
    contact = contact_strvar.get()
    gender = gender_strvar.get()
    DOB = dob.get_date()
    stream = stream_strvar.get()

    if not name or not email or not contact or not gender or not DOB or not stream:
        mb.showerror('Error!', "Please fill all the missing fields!!")
    else:
        try:
            connector.execute(
            'INSERT INTO SCHOOL_MANAGEMENT (NAME, EMAIL, PHONE_NO, GENDER, DOB, STREAM) VALUES (?,?,?,?,?,?)', (name, email, contact, gender, DOB, stream)
            )
            connector.commit()
            mb.showinfo('Record added', f"Record of {name} was successfully added")
            reset_fields()
            display_records()
        except:
            mb.showerror('Wrong type', 'The type of the values entered is not accurate. Pls note that the contact field can only contain numbers')


def remove_record():
    if not tree.selection():
        mb.showerror('Error!', 'Please select an item from the database')
    else:
        current_item = tree.focus()
        values = tree.item(current_item)
        selection = values["values"]

        tree.delete(current_item)

        connector.execute('DELETE FROM SCHOOL_MANAGEMENT WHERE STUDENT_ID=%d' % selection[0])
        connector.commit()

        mb.showinfo('Done', 'The record you wanted deleted was successfully deleted.')

        display_records()


def view_record():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    current_item = tree.focus()
    values = tree.item(current_item)
    selection = values["values"]

    date = datetime.date(int(selection[5][:4]), int(selection[5][5:7]), int(selection[5][8:]))

    name_strvar.set(selection[1]); email_strvar.set(selection[2])
    contact_strvar.set(selection[3]); gender_strvar.set(selection[4])
    dob.set_date(date); stream_strvar.set(selection[6])


# Initializing the GUI window
main = Tk()
main.title('DataFlair School Management System')
main.state('zoomed')
main.resizable(0, 0)

# Creating the background and foreground color variables
lf_bg = 'MediumSpringGreen' # bg color for the left_frame
cf_bg = 'PaleGreen' # bg color for the center_frame

# Creating the StringVar or IntVar variables
name_strvar = StringVar()
email_strvar = StringVar()
contact_strvar = StringVar()
gender_strvar = StringVar()
stream_strvar = StringVar()

# Placing the components in the main window
Label(main, text="SCHOOL MANAGEMENT SYSTEM", font=headlabelfont, bg='SpringGreen').pack(side=TOP, fill=X)

left_frame = Frame(main, bg=lf_bg)
left_frame.place(x=0, y=30, relheight=1, relwidth=0.2)

center_frame = Frame(main, bg=cf_bg)
center_frame.place(relx=0.2, y=30, relheight=1, relwidth=0.2)

right_frame = Frame(main, bg="Gray35")
right_frame.place(relx=0.4, y=30, relheight=1, relwidth=0.6)

# Placing components in the left frame
Label(left_frame, text="Name", font=labelfont, bg=lf_bg).place(relx=0.375, rely=0.05)
Label(left_frame, text="Contact Number", font=labelfont, bg=lf_bg).place(relx=0.175, rely=0.18)
Label(left_frame, text="Email Address", font=labelfont, bg=lf_bg).place(relx=0.2, rely=0.31)
Label(left_frame, text="Gender", font=labelfont, bg=lf_bg).place(relx=0.3, rely=0.44)
Label(left_frame, text="Date of Birth (DOB)", font=labelfont, bg=lf_bg).place(relx=0.1, rely=0.57)
Label(left_frame, text="Stream", font=labelfont, bg=lf_bg).place(relx=0.3, rely=0.7)

Entry(left_frame, width=19, textvariable=name_strvar, font=entryfont).place(x=20, rely=0.1)
Entry(left_frame, width=19, textvariable=contact_strvar, font=entryfont).place(x=20, rely=0.23)
Entry(left_frame, width=19, textvariable=email_strvar, font=entryfont).place(x=20, rely=0.36)
Entry(left_frame, width=19, textvariable=stream_strvar, font=entryfont).place(x=20, rely=0.75)

OptionMenu(left_frame, gender_strvar, 'Male', "Female").place(x=45, rely=0.49, relwidth=0.5)

dob = DateEntry(left_frame, font=("Arial", 12), width=15)
dob.place(x=20, rely=0.62)

Button(left_frame, text='Submit and Add Record', font=labelfont, command=add_record, width=18).place(relx=0.025, rely=0.85)

# Placing components in the center frame
Button(center_frame, text='Delete Record', font=labelfont, command=remove_record, width=15).place(relx=0.1, rely=0.25)
Button(center_frame, text='View Record', font=labelfont, command=view_record, width=15).place(relx=0.1, rely=0.35)
Button(center_frame, text='Reset Fields', font=labelfont, command=reset_fields, width=15).place(relx=0.1, rely=0.45)
Button(center_frame, text='Delete database', font=labelfont, command=reset_form, width=15).place(relx=0.1, rely=0.55)

# Placing components in the right frame
Label(right_frame, text='Students Records', font=headlabelfont, bg='DarkGreen', fg='LightCyan').pack(side=TOP, fill=X)

tree = ttk.Treeview(right_frame, height=100, selectmode=BROWSE,
                    columns=('Student ID', "Name", "Email Address", "Contact Number", "Gender", "Date of Birth", "Stream"))

X_scroller = Scrollbar(tree, orient=HORIZONTAL, command=tree.xview)
Y_scroller = Scrollbar(tree, orient=VERTICAL, command=tree.yview)

X_scroller.pack(side=BOTTOM, fill=X)
Y_scroller.pack(side=RIGHT, fill=Y)

tree.config(yscrollcommand=Y_scroller.set, xscrollcommand=X_scroller.set)

tree.heading('Student ID', text='ID', anchor=CENTER)
tree.heading('Name', text='Name', anchor=CENTER)
tree.heading('Email Address', text='Email ID', anchor=CENTER)
tree.heading('Contact Number', text='Phone No', anchor=CENTER)
tree.heading('Gender', text='Gender', anchor=CENTER)
tree.heading('Date of Birth', text='DOB', anchor=CENTER)
tree.heading('Stream', text='Stream', anchor=CENTER)

tree.column('#0', width=0, stretch=NO)
tree.column('#1', width=40, stretch=NO)
tree.column('#2', width=140, stretch=NO)
tree.column('#3', width=200, stretch=NO)
tree.column('#4', width=80, stretch=NO)
tree.column('#5', width=80, stretch=NO)
tree.column('#6', width=80, stretch=NO)
tree.column('#7', width=150, stretch=NO)

tree.place(y=30, relwidth=1, relheight=0.9, relx=0)

display_records()

# Finalizing the GUI window
main.update()
main.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\syed mubarak pasha\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\syed mubarak pasha\AppData\Local\Temp\ipykernel_13776\2992573202.py", line 97, in view_record
    date = datetime.date(int(selection[5][:4]), int(selection[5][5:7]), int(selection[5][8:]))
IndexError: string index out of range


In [2]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import csv
from tkinter import *
import csv

class StudentManagementSystem(tk.Tk):
    def __init__(self):
        super().__init__()

        # Set window title and size
        self.title("Student Management System")
        self.geometry("800x600")

        # Create a label for the heading
        self.heading_label = ttk.Label(self, text="Student Management System", font=("Helvetica", 18))
        self.heading_label.pack(pady=10)

        # Create a frame to hold the student form fields
        self.student_form_frame = ttk.Frame(self)
        self.student_form_frame.pack(pady=10)

        # Create the form fields for student data
        self.reg_no_label = ttk.Label(self.student_form_frame, text="Register No:")
        self.reg_no_label.grid(row=2, column=0, padx=5, pady=5)
        self.reg_no_entry = ttk.Entry(self.student_form_frame)
        self.reg_no_entry.grid(row=2, column=1, padx=5, pady=5)
        
        self.name_label = ttk.Label(self.student_form_frame, text="Name:")
        self.name_label.grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(self.student_form_frame)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)
        
        self.father_name_label = ttk.Label(self.student_form_frame, text="Father's Name:")
        self.father_name_label.grid(row=3, column=0, padx=5, pady=5)
        self.father_name_entry = ttk.Entry(self.student_form_frame)
        self.father_name_entry.grid(row=3, column=1, padx=5, pady=5)

        self.mother_name_label = ttk.Label(self.student_form_frame, text="Mother's Name:")
        self.mother_name_label.grid(row=4, column=0, padx=5, pady=5)
        self.mother_name_entry = ttk.Entry(self.student_form_frame)
        self.mother_name_entry.grid(row=4, column=1, padx=5, pady=5)

        self.dob_label = ttk.Label(self.student_form_frame, text="Date of Birth:")
        self.dob_label.grid(row=1, column=0, padx=5, pady=5)
        self.dob_entry = ttk.Entry(self.student_form_frame)
        self.dob_entry.grid(row=1, column=1, padx=5, pady=5)

        self.doj_label = ttk.Label(self.student_form_frame, text="Date of Joining:")
        self.doj_label.grid(row=5, column=0, padx=5, pady=5)
        self.doj_entry = ttk.Entry(self.student_form_frame)
        self.doj_entry.grid(row=5, column=1, padx=5, pady=5)

        # Create a button to save student data
        self.save_button = ttk.Button(self, text="Save", command=self.save_student_data)
        self.save_button.pack(pady=10)

    def save_student_data(self):
        # Get the values from the form fields
        reg_no = self.reg_no_entry.get().strip()
        name = self.name_entry.get().strip()
        father_name = self.father_name_entry.get().strip()
        mother_name = self.mother_name_entry.get().strip()
        dob = self.dob_entry.get().strip()
        doj = self.doj_entry.get().strip()

        # Validate the form fields
        if not name:
            messagebox.showerror

fields = ['Reg No', 'First Name', 'Last Name','Mother Name', 'Father Name', 'Email', 'Phone', 'DOB', 'DOJ']

def add_student(entries):
    with open('students.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([entries['Reg No'].get(), entries['First Name'].get(), entries['Last Name'].get(), 
                         entries['Mother Name'].get(), entries['Father Name'].get(), entries['Email'].get(),
                         entries['Phone'].get(), entries['DOB'].get(), entries['DOJ'].get()])

def clear_entries(entries):
    for entry in entries.values():
        entry.delete(0, END)

def makeform(root, fields):
    entries = {}
    for field in fields:
        row = Frame(root)
        lab = Label(row, width=15, text=field, anchor='w')
        ent = Entry(row)
        row.pack(side=TOP, padx=5, pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT, expand=YES, fill=X)
        entries[field] = ent
    return entries

if __name__ == '__main__':
    root = Tk()
    root.title('Student Management System')

    ents = makeform(root, fields)

    add_button = Button(root, text='Add Student', command=lambda: add_student(ents))
    add_button.pack(side=LEFT, padx=5, pady=5)

    clear_button = Button(root, text='Clear Entries', command=lambda: clear_entries(ents))
    clear_button.pack(side=LEFT, padx=5, pady=5)

    quit_button = Button(root, text='Quit', command=root.quit)
    quit_button.pack(side=LEFT, padx=5, pady=5)

    root.mainloop()
